In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import *

In [2]:
df = pd.read_excel("10000gender.xlsx", index_col=False)
df = np.array(df)
label_ans = {0:'Male', 1:'Female'}

In [3]:
characters = "[]'"
data_set = np.zeros((len(df), 3))
for i in range(len(df)):
    string = str(df[i])
    for k in range(len(characters)):
        string = string.replace(characters[k],"")
    string = string.split(',')
    if string[0] == 'Male':
        data_set[i][0] = 0
    elif string[0] == 'Female':
        data_set[i][0] = 1
    for j in range(2):
        data_set[i][j + 1] = float(string[j + 1])
len(data_set)

10000

In [4]:
train_male = np.zeros((50, 2))
train_female = np.zeros((50, 2))
test_male = np.zeros((25, 2))
test_female = np.zeros((25, 2))
for i in range(50):
    train_male[i][:] = data_set[i][1:] 
    train_female[i][:] = data_set[i + 5000][1:] 
for j in range(25):
    test_male[j][:] = data_set[j + 50][1:] 
    test_female[j][:] = data_set[j + 5025][1:] 

In [19]:
male_mean = np.array([np.mean(train_male, axis=0)])
female_mean = np.array([np.mean(train_female, axis=0)])
male_cov = np.zeros((2, 2))
female_cov = np.zeros((2, 2))
for i in range(50):
    male_cov = male_cov + np.matmul((train_male[i][:] - male_mean).T, (train_male[i][:] - male_mean))
    female_cov = female_cov + np.matmul((train_female[i][:] - female_mean).T, (train_female[i][:] - female_mean))
male_cov = male_cov / 50
female_cov = female_cov / 50

male_cov_val = male_cov[0][0] * male_cov[1][1] - male_cov[0][1] * male_cov[1][0]

female_cov_val = female_cov[0][0] * female_cov[1][1] - female_cov[0][1] * female_cov[1][0]
print(f'male mean:{male_mean}')
print(f'female mean:{female_mean}')
print(f'male covariance matrix:\n{male_cov}')
print(f'female covariance matrix:\n{female_cov}')

male mean:[[ 69.09692481 186.25233486]]
female mean:[[ 63.45807626 131.842167  ]]
male covariance matrix:
[[  7.30682057  46.50672325]
 [ 46.50672325 424.45153323]]
female covariance matrix:
[[  7.43563298  44.86026547]
 [ 44.86026547 386.90483163]]


In [22]:
def model_pre(x, male_mean, female_mean, male_cov_val, female_cov_val):
    male_fun = 1 / (2 * np.pi * male_cov_val) * \
        np.exp((-1 * np.matmul((x - male_mean), (x - male_mean).T) * (1 / male_cov_val)) / 2)
    female_fun = 1 / (2 * np.pi * female_cov_val) * \
        np.exp((-1 * np.matmul((x - female_mean), (x - female_mean).T) * (1 / female_cov_val)) / 2)
    return male_fun / (male_fun + female_fun)

In [23]:
male_pre = []
male_pre_val = []
female_pre = []
female_pre_val = []
for i in range(25):
    result = model_pre(test_male[i][:], male_mean, female_mean, male_cov_val, female_cov_val)
    male_pre_val.append(float(result))
    if result > 0.5:
        male_pre.append(1)
    else:
        male_pre.append(0)
    result = model_pre(test_female[i][:], male_mean, female_mean, male_cov_val, female_cov_val)
    female_pre_val.append(float(result))
    if result > 0.5:
        female_pre.append(0)
    else:
        female_pre.append(1)

In [24]:
print(male_pre_val)

[0.5845570964026374, 0.9307642922004099, 0.6994462448391091, 0.7913847792219223, 0.6474788079980777, 0.5527647382027303, 0.8655185208276429, 0.8839944877480397, 0.9184147329158988, 0.9586804437067856, 0.9211603403774766, 0.9170052118521222, 0.9049069597207756, 0.5779878757823004, 0.7796594210804243, 0.42967693421690817, 0.8226287867935536, 0.9800433428990046, 0.8111402977946094, 0.9083732173093356, 0.8242830660251758, 0.9533834020310035, 0.7476781516008416, 0.5514496493035485, 0.9733628316664209]


In [25]:
print(female_pre_val)

[0.1650347438423874, 0.012893483320728889, 0.2157058038079781, 0.20385779140127602, 0.1088902348363397, 0.23623668311664486, 0.046098929223228426, 0.36042189204440206, 0.08089476138933092, 0.22669662081776198, 0.11544237548701516, 0.1611854098624848, 0.1447851487000934, 0.13589350793440128, 0.049586289062074594, 0.30559127829082117, 0.4860208642805284, 0.06403395027535816, 0.16841025943682644, 0.13662857239447057, 0.41526004922322857, 0.22407342362389687, 0.21921043479565572, 0.7827628279609364, 0.34378011869774566]


In [26]:
print(f'male正確:{sum(male_pre)}')
print(f'female正確:{sum(female_pre)}')

male正確:24
female正確:24


In [27]:
print(f'Accuracy: {(sum(male_pre) + sum(female_pre)) / 50 * 100}%')  #正確率

Accuracy: 96.0%
